In [3]:
# Load Packages - KEEP MINIMAL FOR FISHTANK COMPATIBILITY
import pandas as pd
import numpy as np
import os
import sys
import time
from particle import Particle

In [ ]:
# set locations for working files
if len(sys.argv) != 3:
    print("Usage: python3 iter-params.py <automation_dir> <attpcroot_dir>")
    print('Assuming testing directories')
    automation_dir = '/mnt/analysis/e17023/Adam/GADGET2/'
    attpcroot_dir = '/mnt/analysis/e17023/Adam/ATTPCROOTv2/'
else:
    # Automation directory
    automation_dir = sys.argv[1]
    
    # ATTPCROOTv2 directory
    attpcroot_dir = sys.argv[2]

In [ ]:
def indicator_file(file_type, indicator_directory=automation_dir):
    df = pd.DataFrame([0])
    df.to_csv(indicator_directory + file_type + '.csv', index=False)
    #print(file_type + ' FILE CREATED')

In [4]:
def pdgid_lookup(P):
    lookup = {
        # for short-hand notations of particles
        'p': 2212, # proton
        'proton' : 2212,
        'a': 1000020040, # alpha
        'alpha' : 1000020040,
        'b' : 11, # beta
        'beta' : 11,
        'e' : 11, # electron
        'electron' : 11,
        'g' : 22, # gamma
        'gamma' : 22,
    }
    try:
        int(P) # check if P is an integer (PDGID)
        return P
    except ValueError:
        return lookup[P]
    except KeyError:
        indicator_file('STOP')
        raise Exception('Error: Unsupported particle type specified')

In [ ]:
def energy_to_momentum(energy, particle):
    # input energy in KeV, convert to MeV
    energy = energy/1000
    
    pdgid = pdgid_lookup(particle)
    P = Particle.from_pdgid(pdgid)
    mass = P.mass # MeV/c^2 (rest mass)
    
    momentum = np.sqrt(2*mass*energy)/1000 # GeV/c
    return momentum

In [ ]:
parameters = pd.read_csv(automation_dir + 'simInput/parameters.csv')

In [ ]:
# check for and complete any active simulations

# 0 = queued
# 1 = running in attpcroot
# 2 = completed hdf5, waiting for images
# 3 = completed images, waiting for gif
# 4 = completed gif
# 5 = used in tuning (only in tuning mode)

if not parameters['Sim'].is_unique:
    indicator_file('STOP')
    raise Exception('Simulation names are not unique')

active_sims = parameters[parameters['Status'] == 1]
if len(active_sims) > 0:
    indicator_file('STOP')
    print(active_sims['Sim'].values)
    raise Exception('Simulations marked as active found in queue process, not processed successfully')

In [ ]:
# Determine next simulation to run and mark as active
inactive_sims = parameters[parameters['Status'] == 0]

if len(inactive_sims) == 0:
    indicator_file('STOP')
    parameters.to_csv(automation_dir + 'simInput/parameters.csv', index=False)
    print('All simulations complete')
    sys.exit(0)
    
active_sim = inactive_sims.index[0] # select first inactive simulation in queue
indicator_file('BUILD') # indicate that ATTPCROOT needs to be built
    
parameters.loc[active_sim, 'Status'] = 1
if 'time' in parameters.columns:
    parameters.loc[active_sim, 'time'] = time.time()
    
print(f"{parameters.loc[active_sim, 'Sim']}.h5")

In [ ]:
# run check on parameters to confirm they are valid
# TODO - generally fixes itself by erroring out, but should be fixed to avoid wasting time on bad parameters

In [ ]:
# Modify GADGET.sim.par
with open(automation_dir + 'simInput/templates/GADGET.sim.par', 'r') as file :
    filedata = file.readlines()
    
# replace target parameters
for param in parameters.columns:
    for i, line in enumerate(filedata):
        if param == line.split(':')[0]:
            # Line composition:     param:ptype_t   paramval   # units / comments
            
            ptype = line.split(':')[1].split('_')[0]
            paramval = parameters.loc[active_sim, param]
            filedata[i] = param + ': ' + ptype + '_t     ' + str(paramval) + '     #' + line.split('#')[1]
 
# write file
with open(automation_dir + 'Sims/0/queue/GADGET.sim.par', 'w') as file:
    file.writelines(filedata)


In [ ]:
# Modify AtTPC20MgDecay_pag.cxx Generator
P0 = parameters.loc[active_sim, 'P0']; E0 = parameters.loc[active_sim, 'E0']
P1 = parameters.loc[active_sim, 'P1']; E1 = parameters.loc[active_sim, 'E1']


b2b = False
if str(P1).startswith('-'):
    b2b = True # back-to-back event
    P1 = str(P1)[1:]

with open(automation_dir + 'simInput/templates/AtTPC20MgDecay_pag.cxx', 'r') as file :
    filedata = file.readlines()

for i, line in enumerate(filedata):
    if 'P0' in line.split('//')[-1]:
        filedata[i] = line.split('=')[0] + '= ' + str(pdgid_lookup(P0)) + '; // P0\n'
    if 'E0' in line.split('//')[-1]:
        filedata[i] = line.split('=')[0] + '= ' + str(energy_to_momentum(E0, P0)) + '; // E0\n'
    
    if 'P1' in line.split('//')[-1]:
        filedata[i] = line.split('=')[0] + '= ' + str(pdgid_lookup(P1)) + '; // P1\n'
    if 'E1' in line.split('//')[-1]:
        filedata[i] = line.split('=')[0] + '= ' + str(energy_to_momentum(E1, P1)) + '; // E1\n'
    
    if "Seed" in line.split('//')[-1]:
        if 'Seed' in parameters.columns:
            if parameters.loc[active_sim, 'Seed'] != 0:
                filedata[i] = line.split('(')[0] + '(' + str(parameters.loc[active_sim, 'Seed']) + '); // Seed\n'
    
    if 'Efficiency Fix' in line.split('//')[-1]: # place event origin on opposite side of detector to avoid efficiency issues
        if 'Xb' in parameters.columns:
            if parameters.loc[active_sim, 'Xb'] != 99:
                filedata[i] = '// ' + line # comment out line if Xb is not 99 (default behavior unless specified)
        else:
            filedata[i] = '// ' + line # comment out line if Xb isn't specified (default behavior)
    
    if 'b2b' in line.split('//')[-1]:
        if b2b: # back-to-back event
            if 'thetaAlpha' in line: # reference and flip proton theta
                filedata[i] = line.split('=')[0] + '= -1 * ' + line.split('=')[1].split('(')[0] + '(thetaProton); // b2b\n'
            if 'phiAlpha' in line: # reference proton phi
                filedata[i] = line.split('(')[0] + '(phiProton); // b2b\n'
                
            
    
# write file
with open(automation_dir + 'Sims/0/queue/AtTPC20MgDecay_pag.cxx', 'w') as file:
    file.writelines(filedata)

In [ ]:
# MODIFY Mg20_test_sim.C 
with open(automation_dir + 'simInput/templates/Mg20_test_sim_pag.C', 'r') as file:
    filedata = file.readlines()

# Modify particle momentum
for i, line in enumerate(filedata):
    if 'P0 E0' in line.split('//')[-1]:
        filedata[i] = line.split('(')[0] + '(' + str(E0 / 1000000) + ', 1); // P0 E0\n'

    if 'P1 E1' in line.split('//')[-1]:
        if E1 == 0:
            filedata[i] = '// ' + line
        else:
            filedata[i] = line.split('(')[0] + '(' + str(E1 / 1000000) + ', 1); // P1 E1\n'

# modify particle origin
for i, line in enumerate(filedata):
    if 'bounds' in line.split('//')[-1]:
        Xb1 = -2; Xb2 = 2; Yb1 = -2; Yb2 = 2; Zb1 = 10; Zb2 = 40
        if 'Xb' in parameters.columns:
            Xb1 = -1*parameters.loc[active_sim, 'Xb']
            Xb2 = parameters.loc[active_sim, 'Xb']
        if 'Yb' in parameters.columns:
            Yb1 = -1*parameters.loc[active_sim, 'Yb']
            Yb2 = parameters.loc[active_sim, 'Yb']
        if 'Zb1' in parameters.columns:
            Zb1 = parameters.loc[active_sim, 'Zb1']
        if 'Zb2' in parameters.columns:
            Zb2 = parameters.loc[active_sim, 'Zb2']
            
        filedata[i] = line.split('(')[0] + '(' + str(Xb1) + ', ' + str(Yb1) + ', ' + str(Zb1) + ', ' + str(Xb2) + ', ' + str(Yb2) + ', ' + str(Zb2) + '); // bounds\n'

# modify number of particles
if 'N' in parameters.columns:
    filedata[0] = filedata[0].split('=')[0] + '= ' + str(int(parameters.loc[active_sim, 'N'])) + ',' + filedata[0].split(',')[-1]

# modify directory for accurate locations
#for i, line in enumerate(filedata):
    # test for output file
#    if 'Output file name' in line.split('//')[-1]:
#        filedata[i] = line.split('"')[0] + '"' + attpcroot_dir + 'macro/Simulation/Charge_Dispersion/data/gadgetsim.root"' + line.split('"')[-1] 
#    elif 'Parameter file name' in line.split('//')[-1]:
#        filedata[i] = line.split('"')[0] + '"' + attpcroot_dir + 'macro/Simulation/Charge_Dispersion/data/gadgetpar.root"' + line.split('"')[-1]
#    elif 'GeoFile location' in line.split('//')[-1]:
#        filedata[i] = line.split('"')[0] + '"' + attpcroot_dir + 'macro/Simulation/Charge_Dispersion/data/geofile_proto_full.root"' + line.split('"')[-1]

# write file
with open(automation_dir + 'Sims/0/queue/Mg20_test_sim_pag.C', 'w') as file:
    file.writelines(filedata)

In [ ]:
# Modify rundigi_sim.C
with open(automation_dir + 'simInput/templates/rundigi_sim_CD.C', 'r') as file:
    filedata = file.readlines()

# modify number of particles
for i, line in enumerate(filedata):
    if ' N\n' == line.split('//')[-1]:
        if 'N' in parameters.columns:
            filedata[i] = line.split(',')[0] + ', ' + str(int(parameters.loc[active_sim, 'N'])) + '); // N\n'

# modify Threshold
for i, line in enumerate(filedata):
    if 'Threshold' in line.split('//')[-1]:
        if 'Threshold' in parameters.columns:
            threshold = parameters.loc[active_sim, 'Threshold']
        else:
            threshold = 87

        filedata[i] = line.split('(')[0] + '(' + str(threshold) + '); // Threshold\n'

# modify Charge Dispersion Adjacent Pads
for i, line in enumerate(filedata):
    if 'Charge Dispersion' in line.split('//')[-1]:
        if 'CD' in parameters.columns:
            filedata[i] = line.split('(')[0] + '(' + str(int(parameters.loc[active_sim, 'CD'])) + '); // Charge Dispersion\n'

# Modify directory for accurate locations
#for i, line in enumerate(filedata):
#    if 'gadgetsim root file' in line.split('//')[-1]:
#        filedata[i] = line.split('"')[0] + '"' + attpcroot_dir + 'macro/Simulation/Charge_Dispersion/data/gadgetsim.root"' + line.split('"')[-1]

# write file
with open(automation_dir + 'Sims/0/queue/rundigi_sim_CD.C', 'w') as file:
    file.writelines(filedata)

In [ ]:
# Charge Dispersion in AtPulseGADGET.h
with open(automation_dir + 'simInput/templates/AtPulseGADGET.h', 'r') as file:
    filedata = file.readlines()
for i, line in enumerate(filedata):
    if 'CDH' in line.split('//')[-1]:
        # set value to CDH, or CD if CDH is not specified
        if 'CD' in parameters.columns:
            filedata[i] = line.split('=')[0] + '= ' + str(int(parameters.loc[active_sim, 'CD'])) + '; // CDH\n'
            indicator_file('BUILD')
        
        if 'CDH' in parameters.columns:
            filedata[i] = line.split('=')[0] + '= ' + str(int(parameters.loc[active_sim, 'CDH'])) + '; // CDH\n'
            indicator_file('BUILD')

with open(automation_dir + 'Sims/0/queue/AtPulseGADGET.h', 'w') as file:
    file.writelines(filedata)

In [ ]:
# POSSIBLE FUTURE FILE MODIFICATIONS (NOT CURRENTLY IMPLEMENTED)
# Modify R2HMain.cc
# TODO
# Modify media.geo
# TODO
# Modify GADGET_II.C
# TODO
# Modify GADGET_II_lp.C
# TODO

In [ ]:
# Update parameters.csv
parameters.to_csv(automation_dir + 'simInput/parameters.csv', index=False)